In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
port_path = "/Users/sergio.delacruz/Box Sync/data_science/ITAM-Data-Science-and-Machine-Learning-applied-to-financial-markets/05 deep-learning/Prueba.xlsx"
market = pd.read_excel(port_path, index_col=0)

In [7]:
market = market.pct_change().dropna(axis=0)
market.columns = [col.replace("MF Equity", ) for col in rets.columns]
rests.plot,scactter(x="CEMEXCPO MF Equity", y="MEXBOL INDEX")

NameError: name 'rets' is not defined

In [ ]:
Phi = np.c_[np.ones(len(rets)), np.ones(len(rets)), rets[["", ""]].values]
np.linalg.inv(Phi.T @ Phi)

In [ ]:
I = np.concatenate([np.identity(2)] * 2, axis=1)
I = np.concatenate([I] * 2, axis=0)

In [12]:
Phi = rets.iloc[:, :-1].values
t = rets.iloc[:, [-1]]
N, M = Phi.shape

I = np.concatenate([np.identity(M)] * 2, axis=1)
I = np.concatenate([I] * 2, axis=0)

NameError: name 'rets' is not defined

In [13]:
P = np.ones((N, M))
P = np-c_[P, Phi]

NameError: name 'N' is not defined

In [14]:
W = np.linalg.inv((P.T @ P) * I) @ P.T @ t
W["security"] = np.concatenate([rets.columns[:, -1]] * 2)
W["param"] = ["alpha"] * M + ["beta"] * M
B = W.pivot_table(index="security", columns="param", values="MEXBOL INDEX")

NameError: name 'P' is not defined

In [15]:
from ipywidgets import interact

@interact(security=rets.columns[:, -1])
def plot_capm(security):
    rets.plot.scatter(x=security, y="MEXBOL INDEX")
    sec = rets[security]
    x = np.linspace(sec.min(), sec.max(), 100)
    alpha, beta = B.loc[security]
    plt.title(rf"$\alpha={alpha:0.3f}$ | ")
    plt.plot(x, alpha+beta*x, c="tab:orange", linewidth=4)

NameError: name 'rets' is not defined

## Optimización de los pesos

In [18]:
port_path = "/Users/sergio.delacruz/Box Sync/data_science/ITAM-Data-Science-and-Machine-Learning-applied-to-financial-markets/05 deep-learning/PreciosPortf.xlsx"
data = pd.read_excel(port_path, index_col=0)

rend = data.pct_change().dropna()

rend_ports = rend.mean() * 252
rend_ports

ASURB MF Equity       0.123539
GFNORTEO MF Equity    0.078811
PE&OLES* MF Equity    0.111053
WALMEX* MF Equity     0.082426
IENOVA* MF Equity     0.078997
dtype: float64

In [19]:
rend.cov()

,ASURB MF Equity,GFNORTEO MF Equity,PE&OLES* MF Equity,WALMEX* MF Equity,IENOVA* MF Equity
ASURB MF Equity,0.000224,0.000086,0.000061,0.000036,0.000044
GFNORTEO MF Equity,0.000086,0.000341,0.000094,0.000082,0.000082
PE&OLES* MF Equity,0.000061,0.000094,0.000548,0.000041,0.000045
WALMEX* MF Equity,0.000036,0.000082,0.000041,0.000215,0.000043
IENOVA* MF Equity,0.000044,0.000082,0.000045,0.000043,0.000226


In [20]:
weights = np.random.random(5)
weights /= sum(weights)
weights

array([0.20972061, 0.2415211 , 0.18744592, 0.15838071, 0.20293165])

In [21]:
sum(weights)

1.0

In [24]:
# Rendimiento esperado (anualizado) de portafolio
(rend.mean() * weights).sum() * 252

0.09484547619924605

In [27]:
E_p = rend.mean() @ weights * 252
S_p = np.sqrt(weights.T @ (rend.cov() * 252) @ weights)

In [28]:
E_p, S_p 

(0.09484547619924605, 0.17001953935794992)

In [31]:
def port_rend(weights, r):
    E_p = (rend.mean() * weights).sum() * 252
    return E_p

def port_vol(weights, r):
    S_p = np.sqrt(weights.T @ (rend.cov() * 252) @ weights)
    return S_p

In [32]:
port_rend(weights, rend), port_vol(weights, rend)

(0.09484547619924605, 0.17001953935794992)

In [33]:
weights_sim = np.random.randn(5000, 5)
weights_sim = weights_sim / weights_sim.sum(axis=1, keepdims=True)

rend_sim = np.apply_along_axis(port_rend, 1, weights_sim, r=rend)
vol_sim = np.apply_along_axis(port_vol, 1, weights_sim, r=rend)

In [34]:
sharpe_ratio = rend_sim / vol_sim
plt.scatter(vol_sim, rend_sim, c=sharpe_ratio)
plt.colorbar(label="Sharpe Ratio")
plt.xlabel(r"$\")
plt.ylabel("")

# Creando la frontera eficiente

In [35]:
from scipy.optimize import minimize

In [39]:
N, M = rend.shape
w0 = np.random.randn(M)

def sum_weights(weights):
    return weights.sum() -1

constraints = [
    {"type": "eq", "fun": sum_weights}
]

port0 = minimize(port_vol, w0, constraints=constraints, args=rend)
port0

     fun: 0.15516193336666664
     jac: array([0.15512155, 0.15515504, 0.15541889, 0.15511173, 0.15517628])
 message: 'Optimization terminated successfully.'
    nfev: 64
     nit: 9
    njev: 9
  status: 0
 success: True
       x: array([0.28165874, 0.04415679, 0.09018492, 0.30784731, 0.27615224])

In [41]:
wp0 = port0.x
port_rend(wp0, rend), port_vol(wp0, rend)

(0.09548107421425038, 0.15516193336666664)

# Optimización sujeta a un rendimiento esperado

In [44]:
N, M = rend.shape
w0 = np.random.randn(M)

def sum_weights(weights):
    return weights.sum() -1 # == 0

def rend_esperado(weights, E):
    return port_rend(weights, rend) - E # == 0

e0 = 0.10
constraints = [
    {"type": "eq", "fun": sum_weights},
    {"type": "eq", "fun": lambda weights: rend_esperado(weights, e0)}
]

port1 = minimize(port_vol, w0, constraints=constraints, args=rend)
port1

     fun: 0.15715173079322728
     jac: array([0.17765972, 0.13830475, 0.1668106 , 0.14184901, 0.13884013])
 message: 'Optimization terminated successfully.'
    nfev: 78
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([0.37272639, 0.00599319, 0.10743436, 0.27941877, 0.23442729])